In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
df=spark.createDataFrame([['1','2','3','4']])
df.show()

+---+---+---+---+
| _1| _2| _3| _4|
+---+---+---+---+
|  1|  2|  3|  4|
+---+---+---+---+



In [ ]:
sc=spark.sparkContext
df = sc.parallelize([2,3,4,5,6])
df.collect()

[2, 3, 4, 5, 6]

In [ ]:
ss=spark.read.option("header","True").csv("/content/sample_data/mnist_test.csv")
ss.show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+

In [ ]:
ss=spark.read.option("delimiter","|").option("header","True").option("inferschema","True").csv("name.csv")
ss.show()
ss.printSchema()

+---------+----+-----+
|     Name|e_id|value|
+---------+----+-----+
|Tarandeep| e20|   20|
|     Yash| e10|   10|
+---------+----+-----+

root
 |-- Name: string (nullable = true)
 |-- e_id: string (nullable = true)
 |-- value: integer (nullable = true)



In [ ]:
pq=spark.read.parquet("MT cars.parquet")
pq.show()
pq.printSchema()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|          Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|         Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|           Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|
|          M

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
# Create SparkSession
spark = SparkSession.builder \
    .appName("Average, Mean, Sum with Partition") \
    .getOrCreate()

# Sample data
data = [("A", 10), ("A", 20), ("B", 30), ("B", 40), ("B", 50)]

df = spark.createDataFrame(data, ["cyl", "hp"])

# Define window partitioned by 'cyl'
window_spec = Window.partitionBy("cyl")

# Calculate average, min, max, and sum using window functions
wdfn = df.withColumn("avg", F.avg("hp").over(window_spec)) \
    .withColumn("min", F.min("hp").over(window_spec)) \
    .withColumn("max", F.max("hp").over(window_spec)) \
    .withColumn("sum", F.sum("hp").over(window_spec))

# Show result
wdfn.show()
wdfn.select('cyl','sum').dropDuplicates().show()

+---+---+----+---+---+---+
|cyl| hp| avg|min|max|sum|
+---+---+----+---+---+---+
|  B| 30|40.0| 30| 50|120|
|  B| 40|40.0| 30| 50|120|
|  B| 50|40.0| 30| 50|120|
|  A| 10|15.0| 10| 20| 30|
|  A| 20|15.0| 10| 20| 30|
+---+---+----+---+---+---+

+---+---+
|cyl|sum|
+---+---+
|  B|120|
|  A| 30|
+---+---+



In [ ]:
wdfn


DataFrame[cyl: string, hp: bigint, avg: double, min: bigint, max: bigint, sum: bigint]

In [ ]:
from re import MULTILINE
readjson=spark.read.options(MULTILINE=True).json('sample.json')
readjson.show()
readjson.printSchema()

+--------+------------------+------------------+-------------------+
|Adddress|   Job_Description|              Name|             Skills|
+--------+------------------+------------------+-------------------+
|Varanasi|Software Developer|{Tarandeep, Singh}|[Python, Java, SQL]|
| Mathura|Software Developer| {Yatendra, Kumar}|     [Python, Java]|
+--------+------------------+------------------+-------------------+

root
 |-- Adddress: string (nullable = true)
 |-- Job_Description: string (nullable = true)
 |-- Name: struct (nullable = true)
 |    |-- First: string (nullable = true)
 |    |-- Last: string (nullable = true)
 |-- Skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
newl=spark.sparkContext.textFile('/content/sample_data/california_housing_train.csv')
newl.collect()

['"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"',
 '-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000',
 '-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000',
 '-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000',
 '-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000',
 '-114.570000,33.570000,20.000000,1454.000000,326.000000,624.000000,262.000000,1.925000,65500.000000',
 '-114.580000,33.630000,29.000000,1387.000000,236.000000,671.000000,239.000000,3.343800,74000.000000',
 '-114.580000,33.610000,25.000000,2907.000000,680.000000,1841.000000,633.000000,2.676800,82400.000000',
 '-114.590000,34.830000,41.000000,812.000000,168.000000,375.000000,158.000000,1.708300,48500.000000',
 '-114.590000,33.610000,34.0000

In [ ]:
newl=newl.flatMap(lambda a:a.split(','))
newl=newl.map(lambda a: (a,1))
newl.reduceByKey(lambda a,b : a+b).collect()

[('"longitude"', 1),
 ('"housing_median_age"', 1),
 ('"total_bedrooms"', 1),
 ('"median_house_value"', 1),
 ('-114.310000', 1),
 ('34.190000', 113),
 ('1283.000000', 33),
 ('472.000000', 69),
 ('1.493600', 1),
 ('66900.000000', 6),
 ('19.000000', 419),
 ('1901.000000', 14),
 ('1129.000000', 19),
 ('1.820000', 1),
 ('80100.000000', 3),
 ('720.000000', 36),
 ('333.000000', 65),
 ('117.000000', 21),
 ('85700.000000', 10),
 ('-114.570000', 2),
 ('14.000000', 359),
 ('1501.000000', 12),
 ('73400.000000', 7),
 ('326.000000', 72),
 ('262.000000', 50),
 ('1.925000', 4),
 ('65500.000000', 4),
 ('33.630000', 38),
 ('29.000000', 390),
 ('236.000000', 68),
 ('671.000000', 38),
 ('33.610000', 40),
 ('25.000000', 473),
 ('680.000000', 28),
 ('1841.000000', 10),
 ('633.000000', 42),
 ('82400.000000', 3),
 ('34.830000', 7),
 ('168.000000', 30),
 ('4789.000000', 2),
 ('1175.000000', 21),
 ('1056.000000', 32),
 ('58400.000000', 6),
 ('1497.000000', 20),
 ('271.000000', 67),
 ('2.190800', 2),
 ('3741.000

In [ ]:
rdd=spark.sparkContext.parallelize([('a',1),('a',4),('b',10),('b',2),('a',3)])
rdd.collect()

[('a', 1), ('a', 4), ('b', 10), ('b', 2), ('a', 3)]

In [ ]:
rdd.aggregateByKey((0,0),lambda x,y:(x[0]+y,x[1]+1),lambda x,y:(x[0]+y[0],x[1]+y[1])).collect()

[('b', (12, 2)), ('a', (8, 3))]

In [ ]:
rdd.aggregateByKey(1,lambda x,y:x+y , lambda x,y:x+y).collect()

[('b', 13), ('a', 10)]

## RDDOperation

Q1 a

In [ ]:
rdd=spark.sparkContext.textFile('/content/rdd operations.txt')
rdd.collect()

['Certainly! Here are practice questions for each of the specified topics:',
 '',
 '1. **Transformation Operations:**',
 '   a. Write a Spark application to read a text file and convert all words to uppercase using the `map()` transformation.',
 '   b. Given an RDD of integers, filter out numbers greater than 10 using the `filter()` transformation.',
 '   c. Concatenate two RDDs containing strings using the `union()` transformation.',
 '   d. Perform a `flatMap()` transformation on an RDD of sentences to split each sentence into words.',
 '   e. Use the `distinct()` transformation to remove duplicate elements from an RDD of integers.',
 '',
 '2. **Action Operations:**',
 '   a. Calculate the sum of all elements in an RDD of integers using the `reduce()` action.',
 '   b. Find the maximum value in an RDD of floats using the `max()` action.',
 '   c. Count the number of elements in an RDD using the `count()` action.',
 '   d. Collect the first 5 elements of an RDD into a local array usin

In [ ]:
rdd=rdd.flatMap(lambda x: x.split())
rdd=rdd.map(lambda a :a.upper())
rdd.collect()

['CERTAINLY!',
 'HERE',
 'ARE',
 'PRACTICE',
 'QUESTIONS',
 'FOR',
 'EACH',
 'OF',
 'THE',
 'SPECIFIED',
 'TOPICS:',
 '1.',
 '**TRANSFORMATION',
 'OPERATIONS:**',
 'A.',
 'WRITE',
 'A',
 'SPARK',
 'APPLICATION',
 'TO',
 'READ',
 'A',
 'TEXT',
 'FILE',
 'AND',
 'CONVERT',
 'ALL',
 'WORDS',
 'TO',
 'UPPERCASE',
 'USING',
 'THE',
 '`MAP()`',
 'TRANSFORMATION.',
 'B.',
 'GIVEN',
 'AN',
 'RDD',
 'OF',
 'INTEGERS,',
 'FILTER',
 'OUT',
 'NUMBERS',
 'GREATER',
 'THAN',
 '10',
 'USING',
 'THE',
 '`FILTER()`',
 'TRANSFORMATION.',
 'C.',
 'CONCATENATE',
 'TWO',
 'RDDS',
 'CONTAINING',
 'STRINGS',
 'USING',
 'THE',
 '`UNION()`',
 'TRANSFORMATION.',
 'D.',
 'PERFORM',
 'A',
 '`FLATMAP()`',
 'TRANSFORMATION',
 'ON',
 'AN',
 'RDD',
 'OF',
 'SENTENCES',
 'TO',
 'SPLIT',
 'EACH',
 'SENTENCE',
 'INTO',
 'WORDS.',
 'E.',
 'USE',
 'THE',
 '`DISTINCT()`',
 'TRANSFORMATION',
 'TO',
 'REMOVE',
 'DUPLICATE',
 'ELEMENTS',
 'FROM',
 'AN',
 'RDD',
 'OF',
 'INTEGERS.',
 '2.',
 '**ACTION',
 'OPERATIONS:**',
 'A.',

In [ ]:
import re
rdd=rdd.flatMap(lambda x: re.split(r'[; |, ()*.`!]',x))

In [ ]:
rdd=rdd.filter(lambda x: x!="").map(lambda x:(x,1))

In [ ]:
s=rdd.reduceByKey(lambda a,b:a+b)
s=s.map(lambda x:(x[1],x[0]))
s.sortByKey(False).collect()

[(34, 'THE'),
 (20, 'A'),
 (20, 'RDD'),
 (19, 'TO'),
 (18, 'USING'),
 (16, 'OF'),
 (13, 'TRANSFORMATION'),
 (11, 'AND'),
 (11, 'AN'),
 (8, 'FOR'),
 (8, 'PAIR'),
 (6, 'C'),
 (6, 'USE'),
 (6, 'B'),
 (6, 'RDDS'),
 (6, 'D'),
 (6, 'E'),
 (6, 'ELEMENTS'),
 (6, 'ACTION'),
 (6, 'IN'),
 (5, 'FUNCTION'),
 (4, 'KEY'),
 (4, 'OPTIMIZE'),
 (4, 'OPERATIONS:'),
 (4, 'ON'),
 (4, 'FROM'),
 (3, 'SPARK'),
 (3, 'TWO'),
 (3, 'PERFORM'),
 (3, 'IMPLEMENT'),
 (3, 'CUSTOM'),
 (3, 'EACH'),
 (3, 'INTEGERS'),
 (3, 'VALUES'),
 (3, 'WITH'),
 (3, 'OR'),
 (3, 'CREATE'),
 (3, 'OPERATIONS'),
 (3, 'FAULT'),
 (2, 'PRACTICE'),
 (2, 'QUESTIONS'),
 (2, 'TEXT'),
 (2, 'REDUCE'),
 (2, 'NUMBER'),
 (2, 'COMBINE'),
 (2, 'CACHE'),
 (2, 'TIME'),
 (2, 'TAKEN'),
 (2, 'PARTITIONS'),
 (2, 'HANDLING'),
 (2, 'EXCEPTIONS'),
 (2, 'TOLERANCE'),
 (2, 'OVERHEAD'),
 (2, 'FILE'),
 (2, 'ALL'),
 (2, 'WORDS'),
 (2, 'FILTER'),
 (2, 'INTO'),
 (2, 'SUM'),
 (2, 'COUNT'),
 (2, '5'),
 (2, 'JOIN'),
 (2, 'BY'),
 (2, 'PERFORMANCE'),
 (2, 'MEASURE'),
 (2, 'D

In [ ]:
s=s.reduceByKey(lambda x,y:x+" "+y)

print(s.collect())
s=s.map(lambda x:(x[0],x[1].split()))

[(2, 'PRACTICE QUESTIONS TEXT REDUCE NUMBER COMBINE CACHE TIME TAKEN PARTITIONS HANDLING EXCEPTIONS TOLERANCE OVERHEAD FILE ALL WORDS FILTER INTO SUM COUNT 5 JOIN BY PERFORMANCE MEASURE DIFFERENT ERROR HANDLE DATA CHECKPOINT'), (8, 'FOR PAIR'), (16, 'OF'), (6, 'C USE B RDDS D E ELEMENTS ACTION IN'), (4, 'KEY OPTIMIZE OPERATIONS: ON FROM'), (34, 'THE'), (20, 'A RDD'), (18, 'USING'), (1, "CERTAINLY ARE 1 UPPERCASE OUT NUMBERS GREATER THAN 10 CONCATENATE FLATMAP SENTENCES SPLIT DISTINCT CALCULATE MAXIMUM VALUE FLOATS LOCAL SAVE BASED THEIR KEYS GROUPBYKEY SORT SORTBYKEY SAME 4 OPTIMIZATION: MEMORY ACTIONS BROADCAST DISTRIBUTE READ-ONLY WORKER ADJUST PARALLELIZE IN-MEMORY COLLECTION EXPERIMENT CONFIGURATIONS GRACEFULLY WHEN READING EXTERNAL SOURCES LIKE HDFS S3 TRY-CATCH BLOCKS MONITOR PROGRESS RESOURCE IDENTIFY TROUBLESHOOT ERRORS RETRY AUTOMATICALLY MECHANISMS ADVANCED PARTITIONER SHUFFLE COMPLEX AGGREGATIONS ZIP PAIRED SERIALIZATION FORMAT STORAGE NETWORK UTILIZE EFFICIENT CONCEPTS APAC

In [ ]:
s=s.sortByKey(False).collect()

In [ ]:
s[0],s[-1]

((34, ['THE']),
 (1,
  ['CERTAINLY',
   'ARE',
   '1',
   'UPPERCASE',
   'OUT',
   'NUMBERS',
   'GREATER',
   'THAN',
   '10',
   'CONCATENATE',
   'FLATMAP',
   'SENTENCES',
   'SPLIT',
   'DISTINCT',
   'CALCULATE',
   'MAXIMUM',
   'VALUE',
   'FLOATS',
   'LOCAL',
   'SAVE',
   'BASED',
   'THEIR',
   'KEYS',
   'GROUPBYKEY',
   'SORT',
   'SORTBYKEY',
   'SAME',
   '4',
   'OPTIMIZATION:',
   'MEMORY',
   'ACTIONS',
   'BROADCAST',
   'DISTRIBUTE',
   'READ-ONLY',
   'WORKER',
   'ADJUST',
   'PARALLELIZE',
   'IN-MEMORY',
   'COLLECTION',
   'EXPERIMENT',
   'CONFIGURATIONS',
   'GRACEFULLY',
   'WHEN',
   'READING',
   'EXTERNAL',
   'SOURCES',
   'LIKE',
   'HDFS',
   'S3',
   'TRY-CATCH',
   'BLOCKS',
   'MONITOR',
   'PROGRESS',
   'RESOURCE',
   'IDENTIFY',
   'TROUBLESHOOT',
   'ERRORS',
   'RETRY',
   'AUTOMATICALLY',
   'MECHANISMS',
   'ADVANCED',
   'PARTITIONER',
   'SHUFFLE',
   'COMPLEX',
   'AGGREGATIONS',
   'ZIP',
   'PAIRED',
   'SERIALIZATION',
   'FORMAT',
  

##Q1 b

In [ ]:
import random
rdd1b=spark.sparkContext.parallelize(random.sample(range(100),10))
rdd1b.collect()

[18, 61, 62, 48, 44, 0, 91, 43, 3, 77]

In [ ]:
rdd1b.filter(lambda x:x>10).collect()

[18, 61, 62, 48, 44, 91, 43, 77]

##Q1 c

In [ ]:
str1="Since I have not used any arguments for sorting by default it sorts in ascending order. This yields the below output in the console"
str2="The naive method to perform this particular task can be shortened using list comprehension. randrange function is used to perform the task of generating the random numbers. "
rstr1=spark.sparkContext.parallelize(str1.split())
rstr2=spark.sparkContext.parallelize(str2.split())
rstr1.union(rstr2).collect()

['Since',
 'I',
 'have',
 'not',
 'used',
 'any',
 'arguments',
 'for',
 'sorting',
 'by',
 'default',
 'it',
 'sorts',
 'in',
 'ascending',
 'order.',
 'This',
 'yields',
 'the',
 'below',
 'output',
 'in',
 'the',
 'console',
 'The',
 'naive',
 'method',
 'to',
 'perform',
 'this',
 'particular',
 'task',
 'can',
 'be',
 'shortened',
 'using',
 'list',
 'comprehension.',
 'randrange',
 'function',
 'is',
 'used',
 'to',
 'perform',
 'the',
 'task',
 'of',
 'generating',
 'the',
 'random',
 'numbers.']

##Q2

You have been given a dataset containing information about sales transactions. Each transaction consists of the following fields: transaction_id, customer_id, product_id, quantity, and amount. Your task is to analyze this dataset using PySpark and answer the following questions:

a)What is the total revenue generated from all transactions?

b)How many unique customers made purchases?

c)Which product generated the highest revenue, and what was the revenue?

d)What is the average quantity of products purchased per transaction?

e)Can you identify any outliers in the dataset? If so, how would you handle them?


In [ ]:
data=[[1,101,'A001',2,20.00]
,[2,102,'B002',1,15.50]
,[3,103,'C003',3,30.75]
,[4,101,'B002',2,31.00]
,[5,104,'A001',1,10.00]
,[6,105,'C003',2,25.50]
,[7,102,'A001',3,30.00]
,[8,103,'B002',1,15.50]
,[9,101,'A001',1,10.00]
,[10,104,'C003',2,25.50]]
schema=['transaction_id','customer_id','product_id','quantity','amount']
data=spark.createDataFrame(data=data,schema=schema)
data.show()
data.printSchema()

+--------------+-----------+----------+--------+------+
|transaction_id|customer_id|product_id|quantity|amount|
+--------------+-----------+----------+--------+------+
|             1|        101|      A001|       2|  20.0|
|             2|        102|      B002|       1|  15.5|
|             3|        103|      C003|       3| 30.75|
|             4|        101|      B002|       2|  31.0|
|             5|        104|      A001|       1|  10.0|
|             6|        105|      C003|       2|  25.5|
|             7|        102|      A001|       3|  30.0|
|             8|        103|      B002|       1|  15.5|
|             9|        101|      A001|       1|  10.0|
|            10|        104|      C003|       2|  25.5|
+--------------+-----------+----------+--------+------+

root
 |-- transaction_id: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- amount: double (nullable = true)



In [ ]:
#a
import pyspark.sql.functions as F
data_rev=data.withColumn('revenue',F.col('quantity')*F.col('amount'))
data_rev.agg(F.sum('revenue')).show()

+------------+
|sum(revenue)|
+------------+
|      437.25|
+------------+



In [ ]:
#b
data.select('customer_id').distinct().show()

+-----------+
|customer_id|
+-----------+
|        103|
|        104|
|        105|
|        101|
|        102|
+-----------+



In [ ]:
#c
data_rev.groupBy('product_id').agg(F.sum('revenue').alias('total_revenue')).orderBy('total_revenue').show()

+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|      B002|         93.0|
|      A001|        150.0|
|      C003|       194.25|
+----------+-------------+



In [ ]:
#d
data_rev.agg(F.avg('quantity')).show()

+-------------+
|avg(quantity)|
+-------------+
|          1.8|
+-------------+



###29 Feb 2023
1)How to convert the index of a PySpark DataFrame into a column?

2)How to get the minimum, 25th percentile, median, 75th percentile, and maximum of a numeric column?

3)How to get frequency counts of unique items in a column?

In [ ]:
#1
df = spark.createDataFrame([
    ("Alice", 1),
    ("Bob", 2),
    ("Charlie", 3)
], ["Name", "Value"])

df.show()

import pyspark.sql.functions as F
from pyspark.sql.window import Window
w=Window.orderBy('Value')
df.withColumn('index',F.row_number().over(w)).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    1|
|    Bob|    2|    2|
|Charlie|    3|    3|
+-------+-----+-----+



In [ ]:
#2
data = [
    ("A", 10), ("B", 20), ("C", 30),
    ("D", 40), ("E", 50), ("F", 15),
    ("G", 28), ("H", 54), ("I", 41),
    ("J", 86)
]
df = spark.createDataFrame(data, ["Name", "Age"])
df.show()
df.agg(F.min('Age').alias('Min'),F.max('Age').alias('Max'),F.percentile_approx('Age',0.25).alias('25th Per'),F.percentile_approx('Age',0.5).alias('Median'),F.percentile_approx('Age',0.75).alias('75th Per')).show()

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+

+---+---+--------+------+--------+
|Min|Max|25th Per|Median|75th Per|
+---+---+--------+------+--------+
| 10| 86|      20|    30|      50|
+---+---+--------+------+--------+



In [ ]:
#3
data=[[1,101,'A001',2,20.00]
,[2,102,'B002',1,15.50]
,[3,103,'C003',3,30.75]
,[4,101,'B002',2,31.00]
,[5,104,'A001',1,10.00]
,[6,105,'C003',2,25.50]
,[7,102,'A001',3,30.00]
,[8,103,'B002',1,15.50]
,[9,101,'A001',1,10.00]
,[10,104,'C003',2,25.50]]
schema=['transaction_id','customer_id','product_id','quantity','amount']
data=spark.createDataFrame(data=data,schema=schema)
data.show()
data.groupBy('product_id').count().alias('Nymber').show()


+--------------+-----------+----------+--------+------+
|transaction_id|customer_id|product_id|quantity|amount|
+--------------+-----------+----------+--------+------+
|             1|        101|      A001|       2|  20.0|
|             2|        102|      B002|       1|  15.5|
|             3|        103|      C003|       3| 30.75|
|             4|        101|      B002|       2|  31.0|
|             5|        104|      A001|       1|  10.0|
|             6|        105|      C003|       2|  25.5|
|             7|        102|      A001|       3|  30.0|
|             8|        103|      B002|       1|  15.5|
|             9|        101|      A001|       1|  10.0|
|            10|        104|      C003|       2|  25.5|
+--------------+-----------+----------+--------+------+

+----------+-----+
|product_id|count|
+----------+-----+
|      B002|    3|
|      C003|    3|
|      A001|    4|
+----------+-----+



In [ ]:
dd=data.createOrReplaceTempView('newTV')
spark.sql('select product_id,count(*) as Number from newTV group by product_id').show()

+----------+------+
|product_id|Number|
+----------+------+
|      B002|     3|
|      C003|     3|
|      A001|     4|
+----------+------+



A Crash Couse in PySpark :Challenge

In [ ]:
df=spark.read.csv('challenge.csv',header=True,inferSchema=True)
df.show()
df.printSchema()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [ ]:
df.describe().show()

+-------+------------+-----------+-------------+------------------+
|summary|  ip_address|    Country|  Domain Name|        Bytes_used|
+-------+------------+-----------+-------------+------------------+
|  count|        1000|       1000|         1000|              1000|
|   mean|        null|       null|         null|           514.369|
| stddev|        null|       null|         null|287.66708047357156|
|    min|0.23.109.109|Afghanistan|123-reg.co.uk|                 1|
|    max|99.73.193.76|     Zambia|   zimbio.com|              1000|
+-------+------------+-----------+-------------+------------------+



In [ ]:
# add column to add if country is mexico
import pyspark.sql.functions as F
df=df.withColumn('Mexico',F.when(df.Country=='Mexico','Yes').otherwise('No'))
df.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|Mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    No|
| 119.239.207.13|         China|         youtu.be|        51|    No|
|  68.69.217.210|         China|        adobe.com|        10|    No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    No|
| 127.242.24.138|         China| surveymonkey.com|       123|    No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    No|
|   237.54.11.63|         China|       amazon.com|        83|    No|
| 252.141.157.25|         Japan|      cornell.edu|       374|    No|
|185.220.128.248|       Belgium|  

In [ ]:
#2 group by new cal and sum used byte
df.groupby('Mexico').agg(F.sum('Bytes_used').alias('Total_Bytes_used')).show()

+------+----------------+
|Mexico|Total_Bytes_used|
+------+----------------+
|    No|          508076|
|   Yes|            6293|
+------+----------------+



In [ ]:
#3.df distanct ip used by country
df.groupby('Country').agg(F.countDistinct('ip_address').alias('D_ip')).show()

+-----------+----+
|    Country|D_ip|
+-----------+----+
|       Chad|   1|
|     Russia|  56|
|   Paraguay|   1|
|      Yemen|   1|
|     Sweden|  28|
|Philippines|  65|
|   Malaysia|   5|
|     Turkey|   1|
|     Malawi|   2|
|    Germany|   5|
|    Comoros|   1|
|Afghanistan|   5|
|     Rwanda|   1|
|      Sudan|   1|
|     France|  21|
|     Greece|   8|
|  Sri Lanka|   3|
|   Dominica|   1|
|  Argentina|  14|
|    Belgium|   1|
+-----------+----+
only showing top 20 rows



A Crash Couse in PySpark :Original data set clean it and give summary of dataset

In [ ]:
df=spark.read.csv('/content/original.csv',header=True,inferSchema=True)
df.show()
df.printSchema()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
df=df.withColumn('int_sal',when(df.Salary.isNotNull(),df.Salary.substr(2,100)).otherwise(0).cast(FloatType()))
df.show()
df.printSchema()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude| int_sal|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199| 62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266

In [ ]:
df=df.dropna(subset=['JobTitle'])

In [ ]:
df.show()
df.describe().show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude| int_sal|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+--------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199| 62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|30101.16|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36|45.1905

In [ ]:
df.groupBy('gender').agg(avg('int_sal').alias('Avg')).show()

+------+------------------+
|gender|               Avg|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [ ]:
df.groupBy('gender','JobTitle').agg(avg('int_sal').alias('Avg')).show() # avg salary earn by different gender in diff JobTitle

+------+--------------------+------------------+
|gender|            JobTitle|               Avg|
+------+--------------------+------------------+
|Female|    Statistician III|     44224.8984375|
|  Male|     Cost Accountant|40284.178466796875|
|Female|         Engineer IV|    67202.28515625|
|Female| Clinical Specialist|47270.971923828125|
|Female|    Dental Hygienist|25959.773111979168|
|Female|Research Assistan...|   35185.919921875|
|Female|  Nurse Practicioner|  58494.6212890625|
|  Male| Geological Engineer|     59347.6765625|
|  Male|            VP Sales| 67520.01497395833|
|Female|Systems Administr...|       77059.21875|
|Female|Automation Specia...|  44904.1357421875|
|  Male|      Civil Engineer|44401.872767857145|
|  Male|     Design Engineer|   64345.154296875|
|  Male|               Nurse|57115.446614583336|
|  Male|Nuclear Power Eng...|    54780.55546875|
|Female|        Food Chemist|   54655.248046875|
|  Male|Human Resources A...|  33218.5556640625|
|  Male|  Environmen

In [ ]:
df.groupBy('gender','JobTitle').agg(count('id').alias('No_Employees')).show(truncate=False) # No of Emp working of diff gender in diff JobTitle

+------+-----------------------------+------------+
|gender|JobTitle                     |No_Employees|
+------+-----------------------------+------------+
|Female|Statistician III             |1           |
|Male  |Cost Accountant              |8           |
|Female|Engineer IV                  |2           |
|Female|Clinical Specialist          |4           |
|Female|Dental Hygienist             |6           |
|Female|Research Assistant IV        |2           |
|Female|Nurse Practicioner           |5           |
|Male  |Geological Engineer          |5           |
|Male  |VP Sales                     |6           |
|Female|Systems Administrator III    |2           |
|Female|Automation Specialist III    |2           |
|Male  |Civil Engineer               |7           |
|Male  |Design Engineer              |9           |
|Male  |Nurse                        |3           |
|Male  |Nuclear Power Engineer       |5           |
|Female|Food Chemist                 |2           |
|Male  |Huma

Problem Statement :Ipl

In [ ]:
from pyspark.sql.functions import *

In [ ]:
ipl_df=spark.read.csv('/content/IPL_Matches_2008_2022.csv',header=True,inferSchema=True)
ipl_df.show()
ipl_df.printSchema()

+-------+-----------+----------+------+-----------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+-------+------+------+---------------+--------------------+--------------------+--------------------+--------------------+
|     ID|       City|      Date|Season|MatchNumber|               Team1|               Team2|               Venue|          TossWinner|TossDecision|SuperOver|         WinningTeam|  WonBy|Margin|method|Player_of_Match|        Team1Players|        Team2Players|             Umpire1|             Umpire2|
+-------+-----------+----------+------+-----------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+-------+------+------+---------------+--------------------+--------------------+--------------------+--------------------+
|1312200|  Ahmedabad|2022-05-29|  2022|      Final|    Rajasthan Royals|      Gujarat Titans|N

In [ ]:
ipl_df=ipl_df.withColumn('TOSS_match_won',when(ipl_df.TossWinner==ipl_df.WinningTeam,1).otherwise(0))
ipl_df.show()

+-------+-----------+----------+------+-----------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+-------+------+------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+
|     ID|       City|      Date|Season|MatchNumber|               Team1|               Team2|               Venue|          TossWinner|TossDecision|SuperOver|         WinningTeam|  WonBy|Margin|method|Player_of_Match|        Team1Players|        Team2Players|             Umpire1|             Umpire2|TOSS_match_won|
+-------+-----------+----------+------+-----------+--------------------+--------------------+--------------------+--------------------+------------+---------+--------------------+-------+------+------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+
|1312200|  Ahmedabad|2022-05-29|  2022|      Fina

In [ ]:
#1)this code shows us total no of match that year and no of match won by team that played the match
ipl_df.groupBy('Season').agg(sum('TOSS_match_won').alias('Team_won_both_match_toss'),count('TOSS_match_won').alias('Total_no_of_matches')).show()

+-------+------------------------+-------------------+
| Season|Team_won_both_match_toss|Total_no_of_matches|
+-------+------------------------+-------------------+
|2009/10|                      31|                 60|
|   2016|                      34|                 60|
|   2012|                      33|                 74|
|   2019|                      36|                 60|
|   2017|                      34|                 59|
|   2014|                      30|                 60|
|   2013|                      36|                 76|
|2020/21|                      25|                 60|
|   2018|                      32|                 60|
|   2009|                      33|                 57|
|   2011|                      38|                 73|
|   2022|                      36|                 74|
|2007/08|                      28|                 58|
|   2021|                      35|                 60|
|   2015|                      28|                 59|
+-------+-

In [ ]:
#2)teams with most wins
ipl_df.groupBy('WinningTeam').agg(count('ID').alias('No_of_match_won')).orderBy(desc('No_of_match_won')).show(truncate=False)

+---------------------------+---------------+
|WinningTeam                |No_of_match_won|
+---------------------------+---------------+
|Mumbai Indians             |131            |
|Chennai Super Kings        |121            |
|Kolkata Knight Riders      |114            |
|Royal Challengers Bangalore|109            |
|Rajasthan Royals           |96             |
|Kings XI Punjab            |88             |
|Sunrisers Hyderabad        |75             |
|Delhi Daredevils           |67             |
|Delhi Capitals             |36             |
|Deccan Chargers            |29             |
|Punjab Kings               |13             |
|Gujarat Lions              |13             |
|Pune Warriors              |12             |
|Gujarat Titans             |12             |
|Rising Pune Supergiant     |10             |
|Lucknow Super Giants       |9              |
|Kochi Tuskers Kerala       |6              |
|Rising Pune Supergiants    |5              |
|NA                         |4    

In [ ]:
#2)teams with most wins
ipl_df.groupBy('WinningTeam').agg(count('ID').alias('No_of_match_won')).orderBy(desc('No_of_match_won')).show(truncate=False)

+---------------------------+---------------+
|WinningTeam                |No_of_match_won|
+---------------------------+---------------+
|Mumbai Indians             |131            |
|Chennai Super Kings        |121            |
|Kolkata Knight Riders      |114            |
|Royal Challengers Bangalore|109            |
|Rajasthan Royals           |96             |
|Kings XI Punjab            |88             |
|Sunrisers Hyderabad        |75             |
|Delhi Daredevils           |67             |
|Delhi Capitals             |36             |
|Deccan Chargers            |29             |
|Punjab Kings               |13             |
|Gujarat Lions              |13             |
|Gujarat Titans             |12             |
|Pune Warriors              |12             |
|Rising Pune Supergiant     |10             |
|Lucknow Super Giants       |9              |
|Kochi Tuskers Kerala       |6              |
|Rising Pune Supergiants    |5              |
|NA                         |4    

In [ ]:
#3)teams with most playoff wins
ipl_df.groupBy('WinningTeam').agg(sum(when(ipl_df.MatchNumber.contains('a'),1).otherwise(0)).alias('No_of_playOff_win')).orderBy(desc('No_of_playOff_win')).show()

+--------------------+-----------------+
|         WinningTeam|No_of_playOff_win|
+--------------------+-----------------+
| Chennai Super Kings|               15|
|      Mumbai Indians|               12|
|Kolkata Knight Ri...|                8|
|Royal Challengers...|                6|
| Sunrisers Hyderabad|                5|
|    Rajasthan Royals|                4|
|      Delhi Capitals|                2|
|     Deccan Chargers|                2|
|      Gujarat Titans|                2|
|     Kings XI Punjab|                1|
|Rising Pune Super...|                1|
|Lucknow Super Giants|                0|
|                  NA|                0|
|Rising Pune Super...|                0|
|Kochi Tuskers Kerala|                0|
|       Gujarat Lions|                0|
|        Punjab Kings|                0|
|       Pune Warriors|                0|
|    Delhi Daredevils|                0|
+--------------------+-----------------+



In [ ]:
# 4)player with most Player_of_Match
ipl_df.groupBy('Player_of_Match').agg(count('ID').alias('No_of_POM_won')).orderBy(desc('No_of_POM_won')).show()

+---------------+-------------+
|Player_of_Match|No_of_POM_won|
+---------------+-------------+
| AB de Villiers|           25|
|       CH Gayle|           22|
|      DA Warner|           18|
|      RG Sharma|           18|
|       MS Dhoni|           17|
|      SR Watson|           16|
|      YK Pathan|           16|
|        V Kohli|           14|
|       SK Raina|           14|
|     KA Pollard|           14|
|      G Gambhir|           13|
|       A Mishra|           12|
|       KL Rahul|           12|
|      AM Rahane|           12|
|     MEK Hussey|           12|
|      SP Narine|           12|
|     AD Russell|           12|
|       V Sehwag|           11|
|       S Dhawan|           11|
|       DR Smith|           11|
+---------------+-------------+
only showing top 20 rows



In [ ]:
#5)No of match won by chasing or defending
ipl_df.groupBy('WonBy').agg(count('ID').alias('No_of_matches')).show()

+---------+-------------+
|    WonBy|No_of_matches|
+---------+-------------+
|     Runs|          423|
|  Wickets|          509|
|NoResults|            4|
|SuperOver|           14|
+---------+-------------+



In [ ]:
#6)to calculate win percentage and no of match won and played by team

In [ ]:
match_won=ipl_df.groupBy('WinningTeam').agg(count('ID').alias('No_of_Match_Won')).orderBy(desc('No_of_Match_Won'))
match_won.show()

+--------------------+---------------+
|         WinningTeam|No_of_Match_Won|
+--------------------+---------------+
|      Mumbai Indians|            131|
| Chennai Super Kings|            121|
|Kolkata Knight Ri...|            114|
|Royal Challengers...|            109|
|    Rajasthan Royals|             96|
|     Kings XI Punjab|             88|
| Sunrisers Hyderabad|             75|
|    Delhi Daredevils|             67|
|      Delhi Capitals|             36|
|     Deccan Chargers|             29|
|       Gujarat Lions|             13|
|        Punjab Kings|             13|
|      Gujarat Titans|             12|
|       Pune Warriors|             12|
|Rising Pune Super...|             10|
|Lucknow Super Giants|              9|
|Kochi Tuskers Kerala|              6|
|Rising Pune Super...|              5|
|                  NA|              4|
+--------------------+---------------+



In [ ]:
t1=ipl_df.select('ID','Team1')
t2=ipl_df.select('ID','Team2')

In [ ]:
t=t1.union(t2)
t=t.groupBy('Team1').agg(count('ID').alias('Match_played'))
t.show()

+--------------------+------------+
|               Team1|Match_played|
+--------------------+------------+
| Sunrisers Hyderabad|         152|
|Lucknow Super Giants|          15|
| Chennai Super Kings|         208|
|      Gujarat Titans|          16|
|Rising Pune Super...|          16|
|     Deccan Chargers|          75|
|Kochi Tuskers Kerala|          14|
|    Rajasthan Royals|         192|
|       Gujarat Lions|          30|
|Royal Challengers...|         226|
|Kolkata Knight Ri...|         223|
|Rising Pune Super...|          14|
|     Kings XI Punjab|         190|
|        Punjab Kings|          28|
|       Pune Warriors|          46|
|    Delhi Daredevils|         161|
|      Delhi Capitals|          63|
|      Mumbai Indians|         231|
+--------------------+------------+



In [ ]:
Win_per_df=match_won.join(t,match_won.WinningTeam==t.Team1)
Win_per_df.show()

+--------------------+---------------+--------------------+------------+
|         WinningTeam|No_of_Match_Won|               Team1|Match_played|
+--------------------+---------------+--------------------+------------+
| Sunrisers Hyderabad|             75| Sunrisers Hyderabad|         152|
|Lucknow Super Giants|              9|Lucknow Super Giants|          15|
| Chennai Super Kings|            121| Chennai Super Kings|         208|
|      Gujarat Titans|             12|      Gujarat Titans|          16|
|Rising Pune Super...|             10|Rising Pune Super...|          16|
|     Deccan Chargers|             29|     Deccan Chargers|          75|
|Kochi Tuskers Kerala|              6|Kochi Tuskers Kerala|          14|
|    Rajasthan Royals|             96|    Rajasthan Royals|         192|
|       Gujarat Lions|             13|       Gujarat Lions|          30|
|Royal Challengers...|            109|Royal Challengers...|         226|
|Kolkata Knight Ri...|            114|Kolkata Knigh

In [ ]:
Win_per_df.withColumn('win_pec',Win_per_df.No_of_Match_Won*100/Win_per_df.Match_played).select('Team1','No_of_Match_Won','Match_played','win_pec').show()


+--------------------+---------------+------------+------------------+
|               Team1|No_of_Match_Won|Match_played|           win_pec|
+--------------------+---------------+------------+------------------+
| Sunrisers Hyderabad|             75|         152|  49.3421052631579|
|Lucknow Super Giants|              9|          15|              60.0|
| Chennai Super Kings|            121|         208| 58.17307692307692|
|      Gujarat Titans|             12|          16|              75.0|
|Rising Pune Super...|             10|          16|              62.5|
|     Deccan Chargers|             29|          75|38.666666666666664|
|Kochi Tuskers Kerala|              6|          14|42.857142857142854|
|    Rajasthan Royals|             96|         192|              50.0|
|       Gujarat Lions|             13|          30|43.333333333333336|
|Royal Challengers...|            109|         226|48.230088495575224|
|Kolkata Knight Ri...|            114|         223| 51.12107623318386|
|Risin

7)to find the number of matches played by each players

In [ ]:
def add_two_list(a,b):
  return a[1:-1]+','+b[1:-1]

udf_add_two_list=udf(add_two_list)
players_df=ipl_df.withColumn('Players_Name',udf_add_two_list(ipl_df.Team1Players,ipl_df.Team2Players))
players_df.show(truncate=False)

+-------+-----------+----------+------+-----------+---------------------------+---------------------------+---------------------------------------------+---------------------------+------------+---------+---------------------------+-------+------+------+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+---------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ID

In [ ]:
players_df=players_df.select('Players_Name')
players_df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Players_Name                                                                                                                                                                                                                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|'YBK Jaiswal', 

In [ ]:
players_df=players_df.withColumn('Players',split('Players_Name',',')).select(explode('Players').alias('Players'))
players_df.show()

+--------------------+
|             Players|
+--------------------+
|       'YBK Jaiswal'|
|        'JC Buttler'|
|         'SV Samson'|
|        'D Padikkal'|
|        'SO Hetmyer'|
|          'R Ashwin'|
|           'R Parag'|
|          'TA Boult'|
|          'OC McCoy'|
| 'M Prasidh Krishna'|
|         'YS Chahal'|
|           'WP Saha'|
|      'Shubman Gill'|
|           'MS Wade'|
|         'HH Pandya'|
|         'DA Miller'|
|         'R Tewatia'|
|       'Rashid Khan'|
|     'R Sai Kishore'|
|       'LH Ferguson'|
+--------------------+
only showing top 20 rows



In [ ]:
players_df.groupBy('Players').agg(count('*').alias('No_of_matches')).orderBy(desc('No_of_matches')).show()

+------------------+-------------+
|           Players|No_of_matches|
+------------------+-------------+
|        'MS Dhoni'|          233|
|      'KD Karthik'|          229|
|       'RA Jadeja'|          209|
|        'SK Raina'|          204|
|      'KA Pollard'|          189|
|       'AT Rayudu'|          186|
|        'R Ashwin'|          183|
|  'AB de Villiers'|          182|
|         'V Kohli'|          181|
|       'RG Sharma'|          177|
|        'S Dhawan'|          174|
|       'YK Pathan'|          172|
|       'PP Chawla'|          165|
| 'Harbhajan Singh'|          163|
|        'DJ Bravo'|          159|
|        'A Mishra'|          154|
|         'B Kumar'|          146|
|       'MK Pandey'|          143|
|        'UT Yadav'|          133|
|       'SP Narine'|          133|
+------------------+-------------+
only showing top 20 rows



In [ ]:
#8)to show max defeat in match
ipl_df.groupBy('WonBy').agg(max('Margin'),min('Margin')).show()

+---------+-----------+-----------+
|    WonBy|max(Margin)|min(Margin)|
+---------+-----------+-----------+
|     Runs|         98|          1|
|  Wickets|          9|          1|
|NoResults|         NA|         NA|
|SuperOver|         NA|         NA|
+---------+-----------+-----------+



In [ ]:
ipl_df.select('method').distinct().show()

+------+
|method|
+------+
|    NA|
|   D/L|
+------+



In [ ]:
# 9)umpires with most umpiring
u1=ipl_df.select('Umpire1')
u1.show()
u2=ipl_df.select('Umpire2')
u2.show()

+--------------------+
|             Umpire1|
+--------------------+
|         CB Gaffaney|
|         CB Gaffaney|
|       J Madanagopal|
|        BNJ Oxenford|
|        AK Chaudhary|
|         Nitin Menon|
|         CB Gaffaney|
|KN Ananthapadmana...|
|            R Pandit|
|         CB Gaffaney|
|    GR Sadashiv Iyer|
|           PG Pathak|
|            R Pandit|
|        AK Chaudhary|
|       J Madanagopal|
|Chirra Ravikanthr...|
|       NA Patwardhan|
|KN Ananthapadmana...|
|         CB Gaffaney|
|         Nitin Menon|
+--------------------+
only showing top 20 rows

+--------------------+
|             Umpire2|
+--------------------+
|         Nitin Menon|
|         Nitin Menon|
|            MA Gough|
|           VK Sharma|
|       NA Patwardhan|
|        Tapan Sharma|
|       NA Patwardhan|
|    GR Sadashiv Iyer|
|            YC Barde|
|            N Pandit|
|         Nitin Menon|
|        Tapan Sharma|
|           VK Sharma|
|KN Ananthapadmana...|
|            N Pandit|
|       

In [ ]:
u=u1.union(u2)
u.groupBy('Umpire1').agg(count('*').alias('No_of_matches')).orderBy(desc('No_of_matches')).show()

+--------------------+-------------+
|             Umpire1|No_of_matches|
+--------------------+-------------+
|              S Ravi|          131|
|        AK Chaudhary|          111|
|     HDPK Dharmasena|           94|
|       C Shamshuddin|           88|
|         Nitin Menon|           83|
|         CB Gaffaney|           69|
|           M Erasmus|           67|
|           CK Nandan|           59|
|        BNJ Oxenford|           57|
|KN Ananthapadmana...|           56|
|          SJA Taufel|           55|
|           RJ Tucker|           52|
|           Asad Rauf|           51|
|         VA Kulkarni|           50|
|           VK Sharma|           46|
|         BR Doctrove|           42|
|         RE Koertzen|           41|
|      RK Illingworth|           39|
|           Aleem Dar|           38|
|            NJ Llong|           37|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
#10)teams most win in a season
S_win=ipl_df.groupby('Season','WinningTeam').agg(count('*').alias('Wins')).orderBy('Season',desc('Wins'))
S_win.show()

+-------+--------------------+----+
| Season|         WinningTeam|Wins|
+-------+--------------------+----+
|2007/08|    Rajasthan Royals|  13|
|2007/08|     Kings XI Punjab|  10|
|2007/08| Chennai Super Kings|   9|
|2007/08|    Delhi Daredevils|   7|
|2007/08|      Mumbai Indians|   7|
|2007/08|Kolkata Knight Ri...|   6|
|2007/08|Royal Challengers...|   4|
|2007/08|     Deccan Chargers|   2|
|   2009|    Delhi Daredevils|  10|
|   2009|Royal Challengers...|   9|
|   2009|     Deccan Chargers|   9|
|   2009| Chennai Super Kings|   8|
|   2009|     Kings XI Punjab|   7|
|   2009|    Rajasthan Royals|   6|
|   2009|      Mumbai Indians|   5|
|   2009|Kolkata Knight Ri...|   3|
|2009/10|      Mumbai Indians|  11|
|2009/10| Chennai Super Kings|   9|
|2009/10|     Deccan Chargers|   8|
|2009/10|Royal Challengers...|   8|
+-------+--------------------+----+
only showing top 20 rows



In [ ]:
S_win.groupBy('Season').agg(first('WinningTeam').alias('WinningTeam'),first('Wins').alias('Wins')).orderBy('Season').show()

+-------+--------------------+----+
| Season|         WinningTeam|Wins|
+-------+--------------------+----+
|2007/08|    Rajasthan Royals|  13|
|   2009|    Delhi Daredevils|  10|
|2009/10|      Mumbai Indians|  11|
|   2011| Chennai Super Kings|  11|
|   2012|Kolkata Knight Ri...|  12|
|   2013|      Mumbai Indians|  13|
|   2014|     Kings XI Punjab|  12|
|   2015|      Mumbai Indians|  10|
|   2016| Sunrisers Hyderabad|  11|
|   2017|      Mumbai Indians|  12|
|   2018| Chennai Super Kings|  11|
|   2019|      Mumbai Indians|  11|
|2020/21|      Mumbai Indians|  11|
|   2021| Chennai Super Kings|  11|
|   2022|      Gujarat Titans|  12|
+-------+--------------------+----+



In [ ]:
#11)teams most win in a season using windows function
from pyspark.sql import Window
w_ipl=Window.partitionBy('Season').orderBy(S_win.Wins.desc())
S_win.withColumn('rank',dense_rank().over(w_ipl)).select('Season','WinningTeam','Wins').where('rank==1').orderBy('Season').show()

+-------+--------------------+----+
| Season|         WinningTeam|Wins|
+-------+--------------------+----+
|2007/08|    Rajasthan Royals|  13|
|   2009|    Delhi Daredevils|  10|
|2009/10|      Mumbai Indians|  11|
|   2011| Chennai Super Kings|  11|
|   2012|Kolkata Knight Ri...|  12|
|   2013|      Mumbai Indians|  13|
|   2014|     Kings XI Punjab|  12|
|   2015| Chennai Super Kings|  10|
|   2015|      Mumbai Indians|  10|
|   2016| Sunrisers Hyderabad|  11|
|   2017|      Mumbai Indians|  12|
|   2018| Chennai Super Kings|  11|
|   2019|      Mumbai Indians|  11|
|2020/21|      Mumbai Indians|  11|
|   2021| Chennai Super Kings|  11|
|   2022|      Gujarat Titans|  12|
+-------+--------------------+----+



Problem Statement :Ipl ball by ball

In [ ]:
from pyspark.sql.functions import *

In [ ]:
ball_ipl=spark.read.csv('/content/IPL_Ball_by_Ball_2008_2022.csv',header=True,inferSchema=True)
ball_ipl.show()
ball_ipl.printSchema()

+-------+-------+-----+----------+-----------+--------------+-----------+----------+-----------+----------+---------+------------+----------------+----------+----+-----------------+----------------+
|     ID|innings|overs|ballnumber|     batter|        bowler|non-striker|extra_type|batsman_run|extras_run|total_run|non_boundary|isWicketDelivery|player_out|kind|fielders_involved|     BattingTeam|
+-------+-------+-----+----------+-----------+--------------+-----------+----------+-----------+----------+---------+------------+----------------+----------+----+-----------------+----------------+
|1312200|      1|    0|         1|YBK Jaiswal|Mohammed Shami| JC Buttler|        NA|          0|         0|        0|           0|               0|        NA|  NA|               NA|Rajasthan Royals|
|1312200|      1|    0|         2|YBK Jaiswal|Mohammed Shami| JC Buttler|   legbyes|          0|         1|        1|           0|               0|        NA|  NA|               NA|Rajasthan Royals|
|1312

In [ ]:
#1)Score card
ball_ipl.groupBy('ID','innings','BattingTeam').agg(sum('total_run').alias('Runs'),sum('isWicketDelivery').alias('Wickets')).orderBy('ID','innings').show()

+------+-------+--------------------+----+-------+
|    ID|innings|         BattingTeam|Runs|Wickets|
+------+-------+--------------------+----+-------+
|335982|      1|Kolkata Knight Ri...| 222|      3|
|335982|      2|Royal Challengers...|  82|     10|
|335983|      1| Chennai Super Kings| 240|      5|
|335983|      2|     Kings XI Punjab| 207|      4|
|335984|      1|    Rajasthan Royals| 129|      8|
|335984|      2|    Delhi Daredevils| 132|      1|
|335985|      1|      Mumbai Indians| 165|      7|
|335985|      2|Royal Challengers...| 166|      5|
|335986|      1|     Deccan Chargers| 110|     10|
|335986|      2|Kolkata Knight Ri...| 112|      5|
|335987|      1|     Kings XI Punjab| 166|      8|
|335987|      2|    Rajasthan Royals| 168|      4|
|335988|      1|     Deccan Chargers| 142|      8|
|335988|      2|    Delhi Daredevils| 143|      1|
|335989|      1| Chennai Super Kings| 208|      5|
|335989|      2|      Mumbai Indians| 202|      7|
|335990|      1|     Deccan Cha

In [ ]:
#2)Score card by players
ball_ipl.groupBy('ID','innings','batter').agg(sum('batsman_run').alias('Runs')).orderBy('ID','innings').show()

+------+-------+---------------+----+
|    ID|innings|         batter|Runs|
+------+-------+---------------+----+
|335982|      1|     SC Ganguly|  10|
|335982|      1|    BB McCullum| 158|
|335982|      1|Mohammad Hafeez|   5|
|335982|      1|      DJ Hussey|  12|
|335982|      1|     RT Ponting|  20|
|335982|      2|      JH Kallis|   8|
|335982|      2|       SB Joshi|   3|
|335982|      2|        V Kohli|   1|
|335982|      2|       W Jaffer|   6|
|335982|      2|        P Kumar|  18|
|335982|      2|       R Dravid|   2|
|335982|      2|       CL White|   6|
|335982|      2|        B Akhil|   0|
|335982|      2|      AA Noffke|   9|
|335982|      2|         Z Khan|   3|
|335982|      2|     MV Boucher|   7|
|335983|      1|       JDP Oram|  13|
|335983|      1|     MEK Hussey| 116|
|335983|      1|       PA Patel|  15|
|335983|      1|       MS Dhoni|   2|
+------+-------+---------------+----+
only showing top 20 rows



In [ ]:
#3)players most run scored
ball_ipl.groupBy('batter').agg(sum('batsman_run').alias('Runs')).orderBy(desc('Runs')).show()

+--------------+----+
|        batter|Runs|
+--------------+----+
|       V Kohli|6634|
|      S Dhawan|6244|
|     DA Warner|5883|
|     RG Sharma|5881|
|      SK Raina|5536|
|AB de Villiers|5181|
|      CH Gayle|4997|
|      MS Dhoni|4978|
|    RV Uthappa|4954|
|    KD Karthik|4377|
|     G Gambhir|4217|
|     AT Rayudu|4190|
|     AM Rahane|4074|
|      KL Rahul|3895|
|     SR Watson|3880|
|     MK Pandey|3657|
|     SV Samson|3526|
|    KA Pollard|3437|
|  F du Plessis|3403|
|     YK Pathan|3222|
+--------------+----+
only showing top 20 rows



In [ ]:
#4)players most Wicket taken
ball_ipl.groupBy('bowler').agg(sum('isWicketDelivery').alias('Wickets')).orderBy(desc('Wickets')).show()
#it includes runout aswell

+---------------+-------+
|         bowler|Wickets|
+---------------+-------+
|       DJ Bravo|    207|
|     SL Malinga|    188|
|       A Mishra|    175|
|       R Ashwin|    174|
|      YS Chahal|    172|
|      SP Narine|    171|
|      PP Chawla|    165|
|        B Kumar|    165|
|      JJ Bumrah|    161|
|Harbhajan Singh|    161|
|       UT Yadav|    153|
|      RA Jadeja|    141|
|  R Vinay Kumar|    127|
| Sandeep Sharma|    123|
|        A Nehra|    121|
|         Z Khan|    119|
|    Rashid Khan|    119|
| Mohammed Shami|    116|
|       AR Patel|    109|
|       K Rabada|    108|
+---------------+-------+
only showing top 20 rows



In [ ]:
#5)players most Wicket taken no runout no retired hurt
ball_ipl.filter((ball_ipl.kind!='run out') & (ball_ipl.kind!='retired hurt')).groupBy('bowler').agg(sum('isWicketDelivery').alias('Wickets')).orderBy(desc('Wickets')).show()

+---------------+-------+
|         bowler|Wickets|
+---------------+-------+
|       DJ Bravo|    183|
|     SL Malinga|    170|
|      YS Chahal|    166|
|       A Mishra|    166|
|       R Ashwin|    157|
|      PP Chawla|    157|
|        B Kumar|    154|
|      SP Narine|    152|
|Harbhajan Singh|    150|
|      JJ Bumrah|    148|
|       UT Yadav|    135|
|      RA Jadeja|    132|
| Sandeep Sharma|    114|
|    Rashid Khan|    112|
|        A Nehra|    106|
|  R Vinay Kumar|    105|
|       K Rabada|    102|
|         Z Khan|    102|
|       AR Patel|    101|
| Mohammed Shami|     99|
+---------------+-------+
only showing top 20 rows



In [ ]:
#6) higest run score by indivitual
ball_ipl.groupBy('ID','batter').agg(sum('batsman_run').alias('Runs')).orderBy(desc('Runs')).show()

+-------+--------------+----+
|     ID|        batter|Runs|
+-------+--------------+----+
| 598027|      CH Gayle| 175|
| 335982|   BB McCullum| 158|
|1304112|     Q de Kock| 140|
| 829795|AB de Villiers| 133|
|1216510|      KL Rahul| 132|
| 980987|AB de Villiers| 129|
|1136602|       RR Pant| 128|
| 548372|      CH Gayle| 128|
| 419137|       M Vijay| 127|
|1082627|     DA Warner| 126|
|1254085|    JC Buttler| 124|
| 734047|      V Sehwag| 122|
| 501206|   PC Valthaty| 120|
|1254061|     SV Samson| 119|
| 501243|      V Sehwag| 119|
| 829785|      CH Gayle| 117|
| 335990|     A Symonds| 117|
|1136620|     SR Watson| 117|
|1304080|    JC Buttler| 116|
| 335983|    MEK Hussey| 116|
+-------+--------------+----+
only showing top 20 rows



In [ ]:
#7)players most Wicket taken no runout
ball_ipl.filter((ball_ipl.kind!='run out')&(ball_ipl.extra_type!='legbyes')&(ball_ipl.extra_type!='byes')&(ball_ipl.kind!='run out') & (ball_ipl.kind!='retired hurt')).groupBy('id','bowler').agg(sum('isWicketDelivery').alias('Wickets'),sum('total_run').alias('Runs')).orderBy(desc('Wickets'),'Runs').show()

+-------+---------------+-------+----+
|     id|         bowler|Wickets|Runs|
+-------+---------------+-------+----+
|1178394|      AS Joseph|      6|  12|
| 336005|  Sohail Tanvir|      6|  14|
| 980979|        A Zampa|      6|  19|
| 392182|       A Kumble|      5|   5|
|1304102|      JJ Bumrah|      5|  10|
| 501229|       I Sharma|      5|  12|
| 501201|     SL Malinga|      5|  13|
|1136585|     AS Rajpoot|      5|  14|
|1254062|     AD Russell|      5|  15|
| 598065|    JP Faulkner|      5|  16|
| 548311|      RA Jadeja|      5|  16|
|1216512|    LH Ferguson|      5|  17|
|1082603|         AJ Tye|      5|  17|
| 336020|       A Mishra|      5|  17|
|1304100|   PWH de Silva|      5|  18|
| 501221|Harbhajan Singh|      5|  18|
| 548323|      SP Narine|      5|  19|
|1082609|        B Kumar|      5|  19|
| 598032|    JP Faulkner|      5|  20|
|1216497|       CV Varun|      5|  20|
+-------+---------------+-------+----+
only showing top 20 rows



In [ ]:
ball_ipl.select('kind').distinct().show(truncate=False)

+---------------------+
|kind                 |
+---------------------+
|stumped              |
|retired out          |
|hit wicket           |
|NA                   |
|bowled               |
|lbw                  |
|obstructing the field|
|caught and bowled    |
|retired hurt         |
|caught               |
|run out              |
+---------------------+



8)Players and no of catches

In [ ]:
caught=ball_ipl.filter((ball_ipl.kind=='caught') | (ball_ipl.kind=='caught and bowled'))
caught.show()

+-------+-------+-----+----------+------------+-----------------+-------------+----------+-----------+----------+---------+------------+----------------+------------+-----------------+-----------------+--------------------+
|     ID|innings|overs|ballnumber|      batter|           bowler|  non-striker|extra_type|batsman_run|extras_run|total_run|non_boundary|isWicketDelivery|  player_out|             kind|fielders_involved|         BattingTeam|
+-------+-------+-----+----------+------------+-----------------+-------------+----------+-----------+----------+---------+------------+----------------+------------+-----------------+-----------------+--------------------+
|1312200|      1|    3|         6| YBK Jaiswal|       Yash Dayal|   JC Buttler|        NA|          0|         0|        0|           0|               1| YBK Jaiswal|           caught|    R Sai Kishore|    Rajasthan Royals|
|1312200|      1|    8|         2|   SV Samson|        HH Pandya|   JC Buttler|        NA|          0|  

In [ ]:
caught=caught.withColumn('caught',when(caught.kind=='caught',caught.fielders_involved).otherwise(caught.bowler))
caught.show()

+-------+-------+-----+----------+------------+-----------------+-------------+----------+-----------+----------+---------+------------+----------------+------------+-----------------+-----------------+--------------------+--------------+
|     ID|innings|overs|ballnumber|      batter|           bowler|  non-striker|extra_type|batsman_run|extras_run|total_run|non_boundary|isWicketDelivery|  player_out|             kind|fielders_involved|         BattingTeam|        caught|
+-------+-------+-----+----------+------------+-----------------+-------------+----------+-----------+----------+---------+------------+----------------+------------+-----------------+-----------------+--------------------+--------------+
|1312200|      1|    3|         6| YBK Jaiswal|       Yash Dayal|   JC Buttler|        NA|          0|         0|        0|           0|               1| YBK Jaiswal|           caught|    R Sai Kishore|    Rajasthan Royals| R Sai Kishore|
|1312200|      1|    8|         2|   SV Sams

In [ ]:
caught.groupBy('caught').agg(count('*').alias('No_of_Catches')).orderBy(desc('No_of_Catches')).show()

+--------------+-------------+
|        caught|No_of_Catches|
+--------------+-------------+
|      MS Dhoni|          135|
|    KD Karthik|          133|
|AB de Villiers|          120|
|      SK Raina|          109|
|    KA Pollard|          103|
|     RG Sharma|           97|
|       V Kohli|           93|
|      S Dhawan|           93|
|    RV Uthappa|           92|
|     RA Jadeja|           89|
|      DJ Bravo|           80|
|       WP Saha|           79|
|     MK Pandey|           78|
|     DA Warner|           76|
|     SV Samson|           73|
|  F du Plessis|           71|
|      PA Patel|           69|
|       NV Ojha|           65|
|     Q de Kock|           65|
|     DA Miller|           64|
+--------------+-------------+
only showing top 20 rows



In [ ]:
#9)Most Stumping
ball_ipl.filter(ball_ipl.kind=='stumped').groupBy('fielders_involved').agg(count('*').alias('Stumped_Wicket')).orderBy(desc('Stumped_Wicket')).show()

+-----------------+--------------+
|fielders_involved|Stumped_Wicket|
+-----------------+--------------+
|         MS Dhoni|            39|
|       KD Karthik|            34|
|       RV Uthappa|            32|
|          WP Saha|            22|
|          RR Pant|            18|
|         PA Patel|            16|
|     AC Gilchrist|            16|
|        Q de Kock|            15|
|        SV Samson|            12|
|          NV Ojha|            10|
|    KC Sangakkara|             9|
|   AB de Villiers|             8|
|         MS Bisla|             7|
|       SP Goswami|             7|
|        KM Jadhav|             7|
|      BB McCullum|             6|
|          AP Tare|             6|
|         KL Rahul|             5|
|        DH Yagnik|             5|
|      YV Takawale|             4|
+-----------------+--------------+
only showing top 20 rows



10)Find highest partnership by the batsmans

In [ ]:
par_udf=udf(lambda x:sorted(x),ArrayType(StringType()))
partner=ball_ipl.withColumn('partners',par_udf(array('batter','non-striker')))
partner.show()

+-------+-------+-----+----------+-----------+--------------+-----------+----------+-----------+----------+---------+------------+----------------+----------+----+-----------------+----------------+--------------------+
|     ID|innings|overs|ballnumber|     batter|        bowler|non-striker|extra_type|batsman_run|extras_run|total_run|non_boundary|isWicketDelivery|player_out|kind|fielders_involved|     BattingTeam|            partners|
+-------+-------+-----+----------+-----------+--------------+-----------+----------+-----------+----------+---------+------------+----------------+----------+----+-----------------+----------------+--------------------+
|1312200|      1|    0|         1|YBK Jaiswal|Mohammed Shami| JC Buttler|        NA|          0|         0|        0|           0|               0|        NA|  NA|               NA|Rajasthan Royals|[JC Buttler, YBK ...|
|1312200|      1|    0|         2|YBK Jaiswal|Mohammed Shami| JC Buttler|   legbyes|          0|         1|        1|   

In [ ]:
partner.groupBy('partners').agg(sum('total_run').alias('Runs')).orderBy(desc('Runs')).show(truncate=False)

+-------------------------+----+
|partners                 |Runs|
+-------------------------+----+
|[AB de Villiers, V Kohli]|3134|
|[CH Gayle, V Kohli]      |2802|
|[DA Warner, S Dhawan]    |2357|
|[G Gambhir, RV Uthappa]  |1906|
|[KL Rahul, MA Agarwal]   |1731|
|[CH Gayle, KL Rahul]     |1633|
|[MS Dhoni, SK Raina]     |1482|
|[PP Shaw, S Dhawan]      |1461|
|[DA Warner, JM Bairstow] |1401|
|[RR Pant, SS Iyer]       |1368|
|[M Vijay, MEK Hussey]    |1367|
|[BB McCullum, DR Smith]  |1363|
|[KA Pollard, RG Sharma]  |1344|
|[MS Dhoni, RA Jadeja]    |1290|
|[Q de Kock, RG Sharma]   |1274|
|[AM Rahane, SR Watson]   |1169|
|[MEK Hussey, SK Raina]   |1168|
|[AT Rayudu, RG Sharma]   |1165|
|[JC Buttler, SV Samson]  |1147|
|[AM Rahane, R Dravid]    |1145|
+-------------------------+----+
only showing top 20 rows



In [ ]:
ball_ipl.filter(ball_ipl.non_boundary==1).show(truncate=False)


+-------+-------+-----+----------+-------------+-------------+----------------+----------+-----------+----------+---------+------------+----------------+----------+----+-----------------+---------------------------+
|ID     |innings|overs|ballnumber|batter       |bowler       |non-striker     |extra_type|batsman_run|extras_run|total_run|non_boundary|isWicketDelivery|player_out|kind|fielders_involved|BattingTeam                |
+-------+-------+-----+----------+-------------+-------------+----------------+----------+-----------+----------+---------+------------+----------------+----------+----+-----------------+---------------------------+
|1304101|1      |11   |2         |DP Conway    |MR Marsh     |S Dube          |NA        |4          |0         |4        |1           |0               |NA        |NA  |NA               |Chennai Super Kings        |
|1304076|1      |2    |6         |JC Buttler   |UT Yadav     |D Padikkal      |NA        |4          |0         |4        |1           |

In [ ]:
#11) batsmen strike rate

In [ ]:
ball_ipl.select('extra_type').distinct().show()

+----------+
|extra_type|
+----------+
|        NA|
|   noballs|
|   penalty|
|     wides|
|      byes|
|   legbyes|
+----------+



In [ ]:
ball_ipl.filter(ball_ipl.extra_type == 'noballs').show()

+-------+-------+-----+----------+---------------+-----------------+-------------+----------+-----------+----------+---------+------------+----------------+-----------+-------+-----------------+--------------------+
|     ID|innings|overs|ballnumber|         batter|           bowler|  non-striker|extra_type|batsman_run|extras_run|total_run|non_boundary|isWicketDelivery| player_out|   kind|fielders_involved|         BattingTeam|
+-------+-------+-----+----------+---------------+-----------------+-------------+----------+-----------+----------+---------+------------+----------------+-----------+-------+-----------------+--------------------+
|1312198|      1|   11|         1|     RM Patidar|     PVD Chameera|    MK Lomror|   noballs|          1|         1|        2|           0|               0|         NA|     NA|               NA|Royal Challengers...|
|1312197|      1|   19|         6|     JC Buttler|       Yash Dayal|      R Parag|   noballs|          1|         1|        2|          

In [ ]:
strikerate=ball_ipl.groupBy('ID','batter').agg(sum('batsman_run').alias('Runs'),count('*').alias('ball_face'),sum(when((ball_ipl.extra_type=='noballs' )|(ball_ipl.extra_type=='wides'),0).otherwise(1)).alias('ball_face_no_extra')).orderBy(desc('Runs'))
strikerate.show()

+-------+--------------+----+---------+------------------+
|     ID|        batter|Runs|ball_face|ball_face_no_extra|
+-------+--------------+----+---------+------------------+
| 598027|      CH Gayle| 175|       69|                65|
| 335982|   BB McCullum| 158|       77|                73|
|1304112|     Q de Kock| 140|       71|                70|
| 829795|AB de Villiers| 133|       61|                58|
|1216510|      KL Rahul| 132|       70|                69|
| 980987|AB de Villiers| 129|       53|                51|
| 548372|      CH Gayle| 128|       62|                61|
|1136602|       RR Pant| 128|       64|                63|
| 419137|       M Vijay| 127|       57|                56|
|1082627|     DA Warner| 126|       60|                59|
|1254085|    JC Buttler| 124|       69|                63|
| 734047|      V Sehwag| 122|       62|                57|
| 501206|   PC Valthaty| 120|       64|                63|
| 501243|      V Sehwag| 119|       58|                5

In [ ]:
strikerate.withColumn('Strike_Rate',strikerate.Runs*100/strikerate.ball_face_no_extra).orderBy(desc('Strike_Rate')).show()

+-------+--------------------+----+---------+------------------+-----------------+
|     ID|              batter|Runs|ball_face|ball_face_no_extra|      Strike_Rate|
+-------+--------------------+----+---------+------------------+-----------------+
|1082609|         Rashid Khan|   6|        1|                 1|            600.0|
|1216531|            N Pooran|   6|        1|                 1|            600.0|
| 548318|       R Vinay Kumar|   6|        1|                 1|            600.0|
|1254072|           RA Jadeja|   6|        1|                 1|            600.0|
|1136604|JPR Scantlebury-S...|   6|        1|                 1|            600.0|
| 734041|             AP Tare|   6|        1|                 1|            600.0|
|1304090|             DR Sams|   6|        1|                 1|            600.0|
| 548313|           MK Pandey|  12|        4|                 2|            600.0|
| 733979|            R Bhatia|   6|        1|                 1|            600.0|
| 50

In [ ]:
#12) batsman scoring 50+ run with highest strike rate
strikerate.filter(strikerate.Runs>49).withColumn('Strike_Rate',strikerate.Runs*100/strikerate.ball_face_no_extra).orderBy(desc('Strike_Rate')).show()

+-------+---------------+----+---------+------------------+------------------+
|     ID|         batter|Runs|ball_face|ball_face_no_extra|       Strike_Rate|
+-------+---------------+----+---------+------------------+------------------+
|1304060|     PJ Cummins|  56|       15|                14|             400.0|
| 734047|       SK Raina|  87|       26|                24|             362.5|
| 734037|      YK Pathan|  72|       22|                22|327.27272727272725|
|1136562|       KL Rahul|  51|       16|                16|            318.75|
|1082636|      SP Narine|  54|       19|                17| 317.6470588235294|
| 980947|     KA Pollard|  51|       20|                17|             300.0|
|1136601|   Ishan Kishan|  62|       22|                21|295.23809523809524|
|1175358|        RR Pant|  78|       28|                27| 288.8888888888889|
|1216541|      HH Pandya|  60|       22|                21| 285.7142857142857|
| 829717|Harbhajan Singh|  64|       27|            

In [ ]:
#13) batsman scoring 100+ run with highest strike rate
strikerate.filter(strikerate.Runs>99).withColumn('Strike_Rate',strikerate.Runs*100/strikerate.ball_face_no_extra).orderBy(desc('Strike_Rate')).show()

+-------+--------------+----+---------+------------------+------------------+
|     ID|        batter|Runs|ball_face|ball_face_no_extra|       Strike_Rate|
+-------+--------------+----+---------+------------------+------------------+
| 598064|     DA Miller| 101|       39|                37|272.97297297297297|
| 419107|     YK Pathan| 100|       40|                37|270.27027027027026|
| 598027|      CH Gayle| 175|       69|                65| 269.2307692307692|
| 980987|AB de Villiers| 129|       53|                51|252.94117647058823|
| 336018| ST Jayasuriya| 114|       53|                48|             237.5|
| 980999|       V Kohli| 113|       51|                48|235.41666666666666|
| 335994|  AC Gilchrist| 109|       50|                47|231.91489361702128|
| 829795|AB de Villiers| 133|       61|                58|229.31034482758622|
| 419137|       M Vijay| 127|       57|                56|226.78571428571428|
| 335990|     A Symonds| 117|       57|                52|      

In [ ]:
#14) Highest kind of wicket
ball_ipl.filter(ball_ipl.kind!='NA').groupBy('kind').agg(count('*').alias('No_of_Wicket')).orderBy(desc('No_of_Wicket')).show(truncate=False)

+---------------------+------------+
|kind                 |No_of_Wicket|
+---------------------+------------+
|caught               |6837        |
|bowled               |1944        |
|run out              |1007        |
|lbw                  |685         |
|stumped              |325         |
|caught and bowled    |323         |
|hit wicket           |14          |
|retired hurt         |13          |
|obstructing the field|2           |
|retired out          |1           |
+---------------------+------------+



In [ ]:
#15)Average run per inning
score_by_inn=ball_ipl.groupBy('ID','innings','BattingTeam').agg(sum('total_run').alias('Runs'),sum('isWicketDelivery').alias('Wickets')).orderBy('ID','innings')
score_by_inn.show()

+------+-------+--------------------+----+-------+
|    ID|innings|         BattingTeam|Runs|Wickets|
+------+-------+--------------------+----+-------+
|335982|      1|Kolkata Knight Ri...| 222|      3|
|335982|      2|Royal Challengers...|  82|     10|
|335983|      1| Chennai Super Kings| 240|      5|
|335983|      2|     Kings XI Punjab| 207|      4|
|335984|      1|    Rajasthan Royals| 129|      8|
|335984|      2|    Delhi Daredevils| 132|      1|
|335985|      1|      Mumbai Indians| 165|      7|
|335985|      2|Royal Challengers...| 166|      5|
|335986|      1|     Deccan Chargers| 110|     10|
|335986|      2|Kolkata Knight Ri...| 112|      5|
|335987|      1|     Kings XI Punjab| 166|      8|
|335987|      2|    Rajasthan Royals| 168|      4|
|335988|      1|     Deccan Chargers| 142|      8|
|335988|      2|    Delhi Daredevils| 143|      1|
|335989|      1| Chennai Super Kings| 208|      5|
|335989|      2|      Mumbai Indians| 202|      7|
|335990|      1|     Deccan Cha

In [ ]:
score_by_inn.groupBy('innings').agg(avg('Runs').alias('Avg_runs'),avg('Wickets').alias('Avg_Wickets')).show()

+-------+------------------+------------------+
|innings|          Avg_runs|       Avg_Wickets|
+-------+------------------+------------------+
|      1| 162.4115789473684|6.0873684210526315|
|      6|              15.0|               0.0|
|      3| 9.785714285714286|1.3571428571428572|
|      5|              11.0|               1.0|
|      4| 8.785714285714286|               0.5|
|      2|149.28270042194092| 5.633966244725738|
+-------+------------------+------------------+



In [ ]:
#16)Average run per team
score_by_inn.groupBy('BattingTeam').agg(avg('Runs').alias('Avg_runs'),avg('Wickets').alias('Avg_Wickets')).show()

+--------------------+------------------+------------------+
|         BattingTeam|          Avg_runs|       Avg_Wickets|
+--------------------+------------------+------------------+
| Sunrisers Hyderabad|151.55769230769232| 5.532051282051282|
|Lucknow Super Giants|169.86666666666667| 5.933333333333334|
| Chennai Super Kings|159.77511961722487|5.2105263157894735|
|      Gujarat Titans|          166.4375|             5.125|
|Rising Pune Super...|           154.375|             5.625|
|     Deccan Chargers|            152.84| 6.453333333333333|
|Kochi Tuskers Kerala|135.78571428571428| 6.142857142857143|
|    Rajasthan Royals|152.99484536082474| 5.912371134020619|
|       Gujarat Lions|156.83870967741936| 6.064516129032258|
|Royal Challengers...|153.66812227074234|5.6943231441048034|
|Kolkata Knight Ri...|150.66519823788545|5.8061674008810575|
|Rising Pune Super...|147.35714285714286| 4.857142857142857|
|     Kings XI Punjab|154.17435897435897| 5.938461538461539|
|        Punjab Kings|16

In [ ]:
#17) total runs and wicket in diffent overs
ball_ipl.groupBy('overs').agg(sum('total_run').alias('Runs'),sum('isWicketDelivery').alias('Wickets')).orderBy('overs').show()

+-----+-----+-------+
|overs| Runs|Wickets|
+-----+-----+-------+
|    0|11686|    395|
|    1|13567|    433|
|    2|15068|    466|
|    3|15679|    470|
|    4|15741|    485|
|    5|15724|    474|
|    6|12659|    385|
|    7|13647|    402|
|    8|14099|    463|
|    9|13971|    424|
|   10|14295|    488|
|   11|14561|    488|
|   12|14694|    487|
|   13|15037|    544|
|   14|15481|    588|
|   15|15732|    643|
|   16|16228|    675|
|   17|16829|    849|
|   18|16250|    862|
|   19|15149|   1130|
+-----+-----+-------+



In [ ]:
#17) Avg runs and wicket in diffent overs
ball_ipl.groupBy('overs').agg(avg('total_run').alias('Runs'),avg('isWicketDelivery').alias('Wickets')).orderBy('overs').show()

+-----+------------------+--------------------+
|overs|              Runs|             Wickets|
+-----+------------------+--------------------+
|    0|  0.96698386429458|0.032685146876292925|
|    1|1.1367406786761625| 0.03627984918307499|
|    2|1.2766245869694146|0.039481487757349824|
|    3|1.3317760978510151| 0.03992185509216003|
|    4| 1.337837837837838|0.041220465748767636|
|    5|1.3416382252559726|0.040443686006825935|
|    6|1.0846542712706708| 0.03298774740810556|
|    7|1.1713157668869625|0.034503476096472405|
|    8|1.2145933838731908|0.039886285320468645|
|    9|1.2066850924166523|0.036621178096389706|
|   10| 1.242287303380551| 0.04240896845398453|
|   11|1.2683797909407666| 0.04250871080139373|
|   12|1.2830946559552916| 0.04252532308767028|
|   13| 1.324262439453985| 0.04790841039189784|
|   14|1.3749888977706723|0.052224886757260856|
|   15|1.4181916523934013| 0.05796448210583251|
|   16|  1.48052185019615| 0.06158197244776936|
|   17|1.5801877934272301| 0.07971830985